# Create redo records

This Jupyter notebook shows how to create a Senzing "redo record".
It assumes a G2 database that is empty.

Essentially the steps are to create very similar records under different data sources,
then delete one of the records.  This produces a "redo record".

## Prepare environment

### Initialize Senzing configuration

Run [senzing-G2ConfigMgr-reference.ipynb](senzing-G2ConfigMgr-reference.ipynb)
to install a Senzing Engine configuration in the database.

### Initialize python environment

In [1]:
import os
import sys
import json

from IPython.display import JSON

### Helper class for JSON rendering

A class for pretty-printing JSON.
Not required by Senzing,
but helps visualize JSON.

In [2]:
def as_dict(json_data):
    if isinstance(json_data, dict):
        return json_data
    elif isinstance(json_data, bytearray):
        return json.loads(json_data.decode())
    elif isinstance(json_data, bytes):
        return json.loads(json_data.decode())
    elif isinstance(json_data, str):
        return json.loads(json_data)
    return {}

### System path

Update system path.

In [3]:
python_path = "{0}/sdk/python".format(
    os.environ.get("SENZING_G2_DIR", "/opt/senzing/g2"))
sys.path.append(python_path)

### Initialize variables

Create variables used for G2Engine.

In [4]:
%run senzing-init-config.ipynb

Stored 'senzing_config_json' (str)
Default config already set
Stored 'config_id_bytearray' (bytearray)


In [5]:
%store -r senzing_config_json

In [6]:
JSON(as_dict(senzing_config_json))

<IPython.core.display.JSON object>

## G2Engine

### Senzing initialization

Create an instance of G2Engine, G2ConfigMgr, and G2Config.

In [7]:
from senzing import G2Config, G2ConfigMgr, G2Engine, G2Exception

In [8]:
g2_engine = G2Engine()
try:
    g2_engine.init(
        "pyG2EngineForRedoRecords",
        senzing_config_json,
        verbose_logging)
except G2Exception as err:
    print(err)

In [9]:
g2_configuration_manager = G2ConfigMgr()
try:


    g2_configuration_manager.init(
        "pyG2ConfigMgrForRedoRecords",
        senzing_config_json,
        verbose_logging)
except G2Exception.G2ModuleGenericException as err:
    print(g2_configuration_manager.getLastException())

In [10]:
g2_config = G2Config()
try:
    g2_config.init(
        "pyG2ConfigForRedoRecords",
        senzing_config_json,
        verbose_logging)
    config_handle = g2_config.create()
except G2Exception.G2ModuleGenericException as err:
    print(g2_config.getLastException())

### primeEngine

In [11]:
try:
    g2_engine.primeEngine()
except G2Exception.G2ModuleGenericException as err:
    print(err)

### Variable initialization

In [12]:
load_id = None

### Create add data source function

Create a data source with a name having the form `TEST_DATA_SOURCE_nnn`.

In [13]:
def add_data_source(datasource_suffix):
    response = bytearray()
    datasource_prefix = "TEST_DATA_SOURCE_"
    datasource_id = "{0}{1}".format(datasource_prefix, datasource_suffix)
    configuration_comment = "Added {}".format(datasource_id)
    g2_config.addDataSource(config_handle, datasource_id, response)
    configuration_bytearray = bytearray()
    return_code = g2_config.save(config_handle, configuration_bytearray)
    configuration_json = configuration_bytearray.decode()
    configuration_id_bytearray = bytearray()
    g2_configuration_manager.addConfig(configuration_json, configuration_comment, configuration_id_bytearray)
    g2_configuration_manager.setDefaultConfigID(configuration_id_bytearray)
    g2_engine.reinitV2(configuration_id_bytearray)

### Create add record function

Create a record with the id having the form `RECORD_nnn`.
**Note:** this is essentially the same record with only the `DRIVERS_LICENSE_NUMBER` modified slightly.

In [14]:
def add_record(record_id_suffix, datasource_suffix):
    datasource_prefix = "TEST_DATA_SOURCE_"
    record_id_prefix = "RECORD_"
    datasource_id = "{0}{1}".format(datasource_prefix, datasource_suffix)
    record_id = "{0}{1}".format(record_id_prefix, record_id_suffix)
    data = {
        "NAMES": [{
            "NAME_TYPE": "PRIMARY",
            "NAME_LAST": "Smith",
            "NAME_FIRST": "John",
            "NAME_MIDDLE": "M"
        }],
        "PASSPORT_NUMBER": "PP11111",
        "PASSPORT_COUNTRY": "US",
        "DRIVERS_LICENSE_NUMBER": "DL1{:04d}".format(record_id_suffix),
        "SSN_NUMBER": "111-11-1111"
    }
    data_as_json = json.dumps(data)
    g2_engine.addRecord(
        datasource_id,
        record_id,
        data_as_json,
        load_id)

## Redo record

### Print data sources

Print the list of currently defined data sources.

In [15]:
try:
    datasources_bytearray = bytearray()
    g2_config.listDataSources(config_handle, datasources_bytearray)
    datasources_dictionary = json.loads(datasources_bytearray.decode())
    
except G2Exception.G2ModuleGenericException as err:
    print(g2_config.getLastException())
    
JSON(as_dict(datasources_dictionary))

<IPython.core.display.JSON object>

### Add data sources and records

In [17]:
try:
    add_data_source("1")
    add_record("1","1")
    add_record("2","1")
    add_data_source("2")
    add_record("3","2")
    add_record("4","2")
    add_data_source("3")
    add_record("5","3")
    add_record("6","3")
except G2Exception as err:
    print(err)

senzing.G2Exception.G2MalformedJsonException: 30121E|JSON Parsing Failure [code=3,offset=0]


### Delete record

Deleting a record will create a "redo record".

In [18]:
try:
    g2_engine.deleteRecord("TEST", "RECORD_5", load_id)
except G2Exception as err:
    print(err)

### Count redo records

The `count_of_redo_records` will show how many redo records are in Senzing's queue of redo records. 

In [19]:
try:
    count_of_redo_records = g2_engine.countRedoRecords()

    print("Number of redo records: {0}".format(count_of_redo_records))
except G2Exception.G2ModuleGenericException as err:
    print(err)

Number of redo records: 0


### Print data sources again

Print the list of currently defined data sources.

In [20]:
try:
    datasources_bytearray = bytearray()
    g2_config.listDataSources(config_handle, datasources_bytearray)
    datasources_dictionary = json.loads(datasources_bytearray.decode())
    
except G2Exception.G2ModuleGenericException as err:
    print(g2_config.getLastException())
    
JSON(as_dict(datasources_dictionary))

<IPython.core.display.JSON object>